In [1]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker 
# from bokeh.palettes import Bright6
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import output_notebook
from bokeh.models import Legend
from bokeh.embed import file_html
from bokeh.resources import CDN
# from bokeh.io import save
from bokeh.plotting import figure, output_file, save
output_notebook()
sns.set()

Loading BokehJS ...

In [2]:
data = pd.read_csv('data/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
focuscrimes = ['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']
data_focus = data[data['Category'].isin(focuscrimes)]

In [3]:
pand = data_focus['Time'].str.split(':', expand=True).iloc[:,0]
counts = np.array([np.unique(pand[data['Category']==i], return_counts=True)[1] for i in focuscrimes]).astype(np.float64)
counts = counts.T/np.sum(counts,axis=1)
df = pd.DataFrame(counts, columns = focuscrimes)
df0 = pd.DataFrame(np.unique(pand), columns = ['Time'])
df = pd.concat([df0, df], axis=1)

In [4]:
# pd.concat([df0, df],axis=1)
df.head()

,Time,WEAPON LAWS,PROSTITUTION,DRIVING UNDER THE INFLUENCE,ROBBERY,BURGLARY,ASSAULT,DRUNKENNESS,DRUG/NARCOTIC,TRESPASS,LARCENY/THEFT,VANDALISM,VEHICLE THEFT,STOLEN PROPERTY,DISORDERLY CONDUCT
0,00,0.053466,0.081813,0.125796,0.054014,0.038236,0.054166,0.084631,0.031565,0.029645,0.039960,0.056443,0.037630,0.047773,0.052255
1,01,0.039945,0.063511,0.114827,0.055740,0.024795,0.049287,0.079816,0.019300,0.020631,0.024648,0.039514,0.025715,0.033712,0.038462
2,02,0.035469,0.047088,0.115357,0.058219,0.026750,0.044222,0.067930,0.015642,0.023497,0.016049,0.037318,0.020352,0.028210,0.034535
3,03,0.022139,0.032907,0.051132,0.035085,0.027617,0.021713,0.026434,0.010601,0.019225,0.010082,0.025140,0.012652,0.022096,0.021748
4,04,0.014902,0.028180,0.018931,0.022711,0.024476,0.013673,0.013525,0.007724,0.014327,0.006561,0.016763,0.009768,0.019651,0.016110


In [56]:
output_file(filename="custom_filename.html", title="Static HTML file")

source = ColumnDataSource(df)
p = figure(x_range=FactorRange(factors=np.unique(pand)), height=800, width = 1300, title="Crime Counts")#,
            # toolbar_location=None, tools="")
bar ={}
for indx,i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Time',  top=i, source=source, width=0.8, bottom = 0, muted=True, muted_alpha=True,
                    fill_color= (np.random.randint(255),np.random.randint(255),np.random.randint(255)))

items = [(i, [bar[i]]) for i in focuscrimes] ### figure where to add it
legend = Legend(items=items, location=(30,-20)) ## figure where to add it
p.add_layout(legend, 'right') 
p.legend.click_policy="hide" ### assigns the click policy (you can try to use ''hide'


show(p) 

In [54]:
from bokeh.embed import file_html
from bokeh.resources import CDN
# from bokeh.io import save
from bokeh.plotting import figure, output_file, save
output_file(filename="custom_filename.html", title="Static HTML file")
# html = file_html(p, CDN, "my plot")
# save(p)

In [ ]:
# Week days
da = data_focus['DayOfWeek']
week_lab = ['Monday','Tuesday','Wednesday','Thursday' ,'Friday','Saturday','Sunday']
data_new = data_focus.copy()
data_new['DayOfWeek'] = pd.Categorical(da, week_lab)
histvis(data_new, week_lab, focuscrimes, 'Day of week', 'DayOfWeek')

In [57]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

output_file("periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           x_range=groups, y_range=list(reversed(periods)),
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

r = p.rect("group", "period", 0.95, 0.95, source=df, fill_alpha=0.6, legend_field="metal",
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": df, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number",
       text_font_size="11px", **text_props)

p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name",
       text_font_size="7px", **text_props)

p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass",
       text_font_size="7px", **text_props)

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"
p.hover.renderers = [r] # only hover element boxes

show(p)
